This code uses the Clingo parser to parse some of the files in the Results folder (original outputs of the original project), and prints all syntax errors to the console.

In [ ]:
import os
from clingo.ast import parse_files

# NOTE GEORGE: I also tried parse_string, but parse_files seemed more helpful. For parse_string you would need exactly one Clingo statement on each line, but in some
# cases, e.g. with choice rules, a single statement can span multiple lines.

def extract_line_number_from_error(msg):
    """
    Try to extract the line number from a Clingo error message.
    """
    # Example: "Results/CTDeepseek:1:34-41: error: syntax error, unexpected <VARIABLE>"
    # The line nr is between the first and second ":" symbol.
    parts = msg.split(":")
    if len(parts) > 2 and parts[1].isdigit():
        return int(parts[1])
    
    return None

def check_file_syntax(filename: str):
    """
    Parse an entire ASP/Clingo file and collect all syntax errors using Clingo's logger.
    Returns a list of error dictionaries (empty if syntax is correct).
    """

    # Read the file too, so we can log the actual program lines in case of an error.
    with open(filename, "r", encoding="utf-8") as f:
        file_lines = f.readlines()

    errors = []
    def collector(_stmt):
        # Not needed here, but required by Clingo API
        pass

    def logger(code, msg):

        line_number = extract_line_number_from_error(msg)
        code_line = file_lines[line_number - 1].strip()
        
        if "FULL PROGRAM" in code_line:
            return  # Don't log this non-informative message; every program has the line "----------------------------FULL PROGRAM----------------------------"

        errors.append({
            "type": str(code).strip(),
            "message": str(msg).strip(),
            "line number": line_number,
            "code": code_line
        })

    try:
        parse_files([filename], collector, logger=logger)
    except RuntimeError as e:
        pass # This error is always generated if an error is found, but not very useful to log "on top".
        # errors.append({"type": "RuntimeError", "message": str(e), "line": None, "col": None})

    return errors

In [70]:
def parse_file_and_print_errors(filename: str):
    """
    Parse the given ASP/Clingo file and print syntax errors if any.
    """
    print(f"\nChecking syntax of: {filename}\n" + "=" * 100)
    errors = check_file_syntax(filename)

    if not errors:
        print("File is syntactically correct.")
        return

    print("Syntax errors found:\n")
    for err in errors:
        print(f"  Line nr: {err['line number']}")
        print(f"  Type   : {err['type']}")
        print(f"  Message: {err['message']}")
        print(f"  Code   : {err['code']}")
        print()

In [ ]:
#
# Core code; specify the files to check here and runs the parser for each file. All errors are printed to console.
#

filenames = (
    "Results/CTDeepseek",
    "Results/CTLlama8B",
    #"Results/ETDeepseek",     # Doesn't exist? Not sure why
    "Results/ETLlama8B",
    "Results/NSDeepseek",
    "Results/NSLlama8B",
    "Results/SSDeepseek",
    "Results/SSLlama8B"
)

for filename in filenames:
    parse_file_and_print_errors(filename)


Checking syntax of: Results/CTDeepseek
File is syntactically correct.

Checking syntax of: Results/CTLlama8B
File is syntactically correct.

Checking syntax of: Results/ETLlama8B
Syntax errors found:

  Line nr: 71
  Type   : MessageCode.RuntimeError
  Message: Results/ETLlama8B:71:198-199: error: syntax error, unexpected =
  Code   : penalty("SameDayExams",same_day_exams(Exam1,Exam2),5) :- student(Student,Exam1), student(Student,Exam2), period(Period1,_,_,_,_), period(Period2,_,_,_,_), same_day(Exam1,Exam2), |Period1-Period2| \= 1.

  Line nr: 73
  Type   : MessageCode.RuntimeError
  Message: Results/ETLlama8B:73:173-174: error: syntax error, unexpected {
  Code   : same_day(Exam1,Exam2) :- student(Student,Exam1), student(Student,Exam2), period(Period1,_,_,_,_), period(Period2,_,_,_,_), date(Date), Date = { period(Period1,_,_,_,_) } = { period(Period2,_,_,_,_) }.

  Line nr: 78
  Type   : MessageCode.RuntimeError
  Message: Results/ETLlama8B:78:157-158: error: syntax error, unexpecte